## Analizing life expectancy and GDP

The goal of this project is to get a better understanding of life expectancy and if and how it is correlated with GDP.

Data provided by: [Codecademy](https://www.codecademy.com) 

*Data Scientist: Machine Learning course, Data Visualization Portfolio Project module.*

`all_data.csv`

### Project scope

#### Goals

Gaining insight on the provided data. As the title says we will look at data containing life expectancy and GDP ad main features, probably more infomation.

#### Analysis

The main focus is exploring the provided data (EDA) with necessary vizualisation in order to finally evaluate it.

0. Importing necessary libraries

In [1]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import seaborn as sns

1. Loading the data

In [2]:
raw_data = pd.read_csv("all_data.csv")
raw_data.head()

,Country,Year,Life expectancy at birth (years),GDP
0,Chile,2000,77.3,7.786093e+10
1,Chile,2001,77.3,7.097992e+10
2,Chile,2002,77.8,6.973681e+10
3,Chile,2003,77.9,7.564346e+10
4,Chile,2004,78.0,9.921039e+10


In [4]:
raw_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 96 entries, 0 to 95
Data columns (total 4 columns):
 #   Column                            Non-Null Count  Dtype  
---  ------                            --------------  -----  
 0   Country                           96 non-null     object 
 1   Year                              96 non-null     int64  
 2   Life expectancy at birth (years)  96 non-null     float64
 3   GDP                               96 non-null     float64
dtypes: float64(2), int64(1), object(1)
memory usage: 3.1+ KB
